# 微调模型手册

以轻量级的 Qwen2.5-0.5B-Instruct 模型为例。

## 第一部分：通用准备工作


在开始任何微调之前，我们需要先完成环境和数据的准备。

### 第 1 步：统一环境搭建
一个稳定且隔离的Python环境是成功的第一步。

使用 Conda 可以创建一个隔离的 Python 环境，防止包版本冲突。

In [ ]:
# # 使用 Conda"
# conda create -n finetune-env python=3.12
# conda activate finetune-env

### 第 2 步：LLaMA-Factory 部署与依赖安装

#### 2.1 设置工作目录

In [ ]:
import os
from pathlib import Path

# 1. 设置你要用的子目录名
project_dir_name = "Workspace"

# 2. 尝试挂载 Google Drive
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    base_dir = Path("/content/drive/MyDrive")  # 云盘根目录
    print("✅ 成功挂载 Google Drive")
except Exception as e:
    base_dir = Path("/content")  # 回退到本地 Colab 空间
    print("⚠️ 无法挂载 Google Drive，使用本地目录")

# 3. 设置工作目录
work_dir = base_dir / project_dir_name
work_dir.mkdir(parents=True, exist_ok=True)  # 自动创建（如果不存在）
os.chdir(work_dir)
print(f"📂 当前工作目录已切换到: {work_dir.resolve()}")

⚠️ 无法挂载 Google Drive，使用本地目录
📂 当前工作目录已切换到: /content/LLaMA-Factory


#### 2.2 克隆 LLaMA-Factory 仓库

In [ ]:
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [ ]:
%cd LLaMA-Factory

[Errno 2] No such file or directory: 'LLaMA-Factory'
/content


#### 2.3 安装核心与高性能依赖

In [ ]:
%nvidia-smi

UsageError: Line magic function `%nvidia-smi` not found.


In [ ]:
# 卸载旧版本包（防止冲突）
# pip uninstall -y torch torchvision torchaudio flash-attn deepspeed llamafactory tensorflow tensorflow-decision-forests dopamine-rl opencv-python opencv-python-headless opencv-contrib-python thinc

# 核心库安装 (请根据您的 CUDA 版本选择合适的 PyTorch 安装命令)
!uv pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

# Hugging Face 生态系统和微调工具
!uv pip install "transformers>=4.41.0,<5.0.0"
!uv pip install sentence-transformers
!uv pip install -q "accelerate>=0.30.1"
!uv pip install -q "peft>=0.10.0"
!uv pip install -q "datasets>=2.19.1"
!uv pip install -q "deepspeed>=0.14.2"
!uv pip install -q "sentence-transformers==4.1.0"

# Qwen-VL 特定库和性能优化库
!uv pip install -q "bitsandbytes>=0.43.1"
!uv pip install -q "qwen-vl-utils>=0.0.11"
# !pip install -q "flash-attn==2.5.8" --no-build-isolation

# 可视化和监控工具
!uv pip install -q "supervision>=0.20.0"
!uv pip install -q "tensorboard==2.18"
!uv pip install -q "wandb>=0.17.0"
!uv pip install -q peft trl


# 安装 Flash Attention（可能需要编译工具 cmake、ninja）
!uv pip install flash-attn --no-build-isolation

# 安装 DeepSpeed（使用阿里镜像加速）
!uv pip install deepspeed -i https://mirrors.aliyun.com/pypi/simple/


# 安装其他依赖
!uv pip install trl modelscope addict

# 可选：验证 GPU 与 PyTorch
!python -c "import torch; print(torch.cuda.is_available(), torch.version.cuda, torch.backends.cudnn.version())"

# !pip install -e .[torch,metrics]

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.24 requires opencv-python-headless>=4.9.0.80, which is not installed.
peft 0.17.0 requires torch>=1.13.0, which is not installed.
spacy 3.8.7 requires thinc<8.4.0,>=8.3.4, which is not installed.
accelerate 1.10.0 requires torch>=2.0.0, which is not installed.
albumentations 2.0.8 requires opencv-python-headless>=4.9.0.80, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
fastai 2.7.19 requires torchvision>=0.11, which is not installed.


Looking in indexes: https://download.pytorch.org/whl/cu126
  Using cached https://download.pytorch.org/whl/cu126/torch-2.8.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (30 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchvision-0.23.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (6.1 kB)
  Using cached https://download.pytorch.org/whl/cu126/torchaudio-2.8.0%2Bcu126-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (7.2 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.13.3-py3-none-any.whl.metadata (12 kB)
  Using cached https://download.pytorch.org/whl/cu126/nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached https://download.pytorch.org/whl/cu126/nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached https://download.pytorch.org/whl/cu126/nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached https://download.

#### 2.4 验证安装 LLaMA-Factory

运行 LLaMA-Factory 的命令行工具，检查是否能成功打印版本号。

In [ ]:
!llamafactory-cli version

[2025-08-16 03:37:15,067] [WARNING] [real_accelerator.py:209:get_accelerator] Setting accelerator to CPU. If you have GPU or other accelerator, we were unable to detect it.
[2025-08-16 03:37:15,069] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cpu (auto detect)
[2025-08-16 03:37:19,988] [INFO] [logging.py:107:log_dist] [Rank -1] [TorchCheckpointEngine] Initialized with serialization = False
----------------------------------------------------------
| Welcome to LLaMA Factory, version 0.9.4.dev0           |
|                                                        |
| Project page: https://github.com/hiyouga/LLaMA-Factory |
----------------------------------------------------------


如果安装成功，应该能看到 LLaMA-Factory 的欢迎信息和版本号。

### 第 3 步：数据准备与自动注册
高质量的数据是模型“学得好”的关键。不同的微调阶段需要不同格式的数据。

#### **3.1 数据格式说明**


| 数据格式 / 字段结构                                          | LLaMA-Factory            | Hugging Face Trainer / PEFT   | Hugging Face TRL（DPO/PPO）    | 说明                              |
| ------------------------------------------------------------ | ------------------------ | ----------------------------- | ------------------------------ | --------------------------------- |
| ✅ `conversations` 列表结构（支持多轮对话）                   | ✅ 原生支持               | ❌ 不支持，需要手动拼接        | ❌ 不支持，需要手动拼接         | 只 LLaMA-Factory 会解析成对话模板 |
| ✅ 支持 `system` 消息                                         | ✅ 支持（参与上下文拼接） | ❌ 不支持（需手动拼入 prompt） | ❌ 不支持（需拼入 prompt）      | Hugging Face 不解析角色字段       |
| ✅ 单轮问答格式： `{"input": "prompt", "output": "response"}` | ❌ 不推荐用此格式         | ✅ 标准支持格式                | ⚠️ 支持（但字段名可能需自定义） | Hugging Face 推荐的监督微调格式   |
| ✅ DPO 格式： `{"instruction": "...", "output": [chosen, rejected]}` | ✅ 支持（train_dpo.py）   | ❌ 不支持                      | ✅ 支持（`DPOTrainer`）         | LLaMA 和 TRL 都支持               |
| ✅ DPO 格式变体： `{"prompt": "...", "chosen": "...", "rejected": "..."}` | ✅ 支持                   | ❌ 不支持                      | ✅ 支持                         | TRL 推荐格式                      |
| ✅ RM 训练格式： `{"prompt": "...", "chosen": "...", "rejected": "..."}` | ✅ 支持                   | ❌ 不支持                      | ⚠️ 部分支持（需自定义 RM 构造） | 用于奖励模型训练                  |
| ✅ GRPO / PPO： `{"prompt": "..."} 或含 history`              | ✅ 支持                   | ❌ 不支持                      | ✅ 支持（`PPOTrainer`）         | history 一般需手动拼入 prompt     |
| ✅ `.json` 文件（数组形式）                                   | ✅ 支持                   | ✅ 支持                        | ✅ 支持                         | JSON 文件均支持                   |
| ✅ `.jsonl` 文件（每行一个 JSON）                             | ✅ 推荐格式               | ✅ 支持                        | ✅ 支持                         | 更适合大规模训练                  |



#### **3.2 公开数据集推荐**

对于许多场景，社区已经贡献了高质量的数据集，可以直接使用。



##### **SFT (监督微调) 数据集**


SFT 阶段的目标是教会模型基础的知识和对话风格。此阶段最常用的格式是 `instruction`/`output` 格式，特别适合指令跟随和单轮问答。

- **第一步：下载数据集** 我们将使用优质数据集 `CFYuan/Chat-huanhuan`，它的格式就是标准的 `instruction`/`input`/`output`。创建一个 `download_sft_dataset.py` 脚本来下载并保存它



In [ ]:
# download_sft_dataset.py
from modelscope.msdatasets import MsDataset
import json

def save_dataset_to_json(dataset, output_path):
    """ 将数据集保存为 JSON Lines 格式 (.jsonl) """
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"数据集已成功保存到: {output_path}")

if __name__ == "__main__":
    # 从 ModelScope 加载数据集
    dataset_name = 'CFYuan/Chat-huanhuan'
    ms_dataset = MsDataset.load(dataset_name, subset_name='default', split='train', trust_remote_code=True)

    # 定义本地保存路径
    output_file_path = "data/huanhuan_sft_local.json"

    # 直接保存
    save_dataset_to_json(ms_dataset, output_file_path)

2025-08-15 09:29:06,489 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from Chat-huanhuan. Please make sure that you can trust the external codes.
2025-08-15 09:29:08,324 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from CFYuan/Chat-huanhuan. Please make sure that you can trust the external codes.
2025-08-15 09:29:08,325 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from CFYuan/Chat-huanhuan. Please make sure that you can trust the external codes.
2025-08-15 09:29:08,326 - modelscope - WARNING - Use trust_remote_code=True. Will invoke codes from CFYuan/Chat-huanhuan. Please make sure that you can trust the external codes.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your 

数据集已成功保存到: data/huanhuan_sft_local.json


运行脚本 python download_sft_dataset.py，数据将被保存到 data/huanhuan_sft_local.json。


- **通用指令型：`BelleGroup/train_2M_CN`**
  - **简介**：由 BELLE 项目生成的、包含约200万条多样化指令的中文数据集。
  - **适用场景**：全面提升模型在中文环境下的通用指令遵循能力（如问答、翻译、写作等）。
  - **LLaMA-Factory 名称**: `belle_2m`

##### DPO/ORPO 数据准备

- **第一步：下载数据集** 我们将使用高质量的中文问答偏好数据集 `liyucheng/zhihu_rlhf_3k`。创建一个 `download_dpo_dataset.py` 脚本来下载并保存它。

In [ ]:
# download_dpo_dataset.py
from datasets import load_dataset
import json

def save_dataset_to_json(dataset, output_path):
    """ 将数据集保存为 JSON Lines 格式 (.jsonl) """
    with open(output_path, 'w', encoding='utf-8') as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"数据集已成功保存到: {output_path}")

if __name__ == "__main__":
    # 从 Hugging Face Hub 加载数据集
    dataset_name = 'liyucheng/zhihu_rlhf_3k'
    dataset = load_dataset(dataset_name, split='train')

    # 定义本地保存路径
    output_file_path = "data/zhihu_rlhf_local.json"

    # 直接保存
    save_dataset_to_json(dataset, output_file_path)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

zhihu_3k_rlfh.tsv:   0%|          | 0.00/15.6M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3460 [00:00<?, ? examples/s]

数据集已成功保存到: data/zhihu_rlhf_local.json


运行脚本 python download_dpo_dataset.py，数据将被保存到 data/zhihu_rlhf_local.json。

为了让模型先学习“知乎问答”的基本模式，我们从 DPO 数据中提取高质量的问答对，作为 SFT 数据。创建一个 create_sft_from_dpo.py 脚本。

In [ ]:
# create_sft_from_dpo.py
import json

def convert_dpo_to_sft(dpo_path, sft_path):
    sft_data = []
    with open(dpo_path, 'r', encoding='utf-8') as f:
        for line in f:
            item = json.loads(line)
            sft_item = {
                "instruction": item["prompt"],
                "input": "",
                "output": item["chosen"] # 使用高质量回答作为SFT的答案
            }
            sft_data.append(sft_item)

    with open(sft_path, 'w', encoding='utf-8') as f:
        for item in sft_data:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')
    print(f"已从 {dpo_path} 提取并创建 SFT 数据集于: {sft_path}")

if __name__ == "__main__":
    convert_dpo_to_sft("data/zhihu_rlhf_local.json", "data/zhihu_sft_local.json")

运行 python create_sft_from_dpo.py。

##### 通过代码注册数据集 (推荐)

下载完数据集后，手动编辑 `dataset_info.json` 较为繁琐。我们可以创建一个脚本来自动完成注册。

- **创建 `register_datasets.py` 脚本** 在 `LLaMA-Factory` 根目录下创建该文件。

In [ ]:
# register_datasets.py
import json
import os

def register_dataset(dataset_info_path, dataset_name, file_name, columns, stage):
    """
    以编程方式读取、更新并保存 dataset_info.json 文件，并包含 stage 信息。
    """
    if os.path.exists(dataset_info_path):
        with open(dataset_info_path, 'r', encoding='utf-8') as f:
            all_datasets = json.load(f)
    else:
        all_datasets = {}

    all_datasets[dataset_name] = {
        "file_name": file_name,
        "stage": stage, # 添加数据集适用的阶段
        "columns": columns,
        "ranking": stage=="rm",
    }

    with open(dataset_info_path, 'w', encoding='utf-8') as f:
        json.dump(all_datasets, f, indent=2, ensure_ascii=False)
    print(f"成功注册数据集 '{dataset_name}' (阶段: {stage})")

if __name__ == "__main__":
    info_path = "data/dataset_info.json"
    # 注册 SFT 章节的数据集
    register_dataset(info_path, "huanhuan_sft_local", "huanhuan_sft_local.json",
                     {"prompt": "instruction", "query": "input", "response": "output"}, stage="sft")
    # 注册 DPO 章节可选 SFT 步骤的数据集
    register_dataset(info_path, "zhihu_sft_local", "zhihu_sft_local.json",
                     {"prompt": "instruction", "query": "input", "response": "output"}, stage="sft")
    # 注册 DPO 章节的数据集
    # DPO 阶段在 LLaMA-Factory 中内部使用 "rm" (Reward Modeling) 的数据加载逻辑
    register_dataset(info_path, "zhihu_dpo_local", "zhihu_rlhf_local.json",
                     {"prompt": "prompt", "chosen": "chosen", "rejected": "rejected"}, stage="rm")

FileNotFoundError: [Errno 2] No such file or directory: 'data/dataset_info.json'

- 运行脚本

执行 python register_datasets.py，即可一次性完成所有本地数据集的注册。

##### GRPO (生成式重放策略优化) 数据准备
GRPO 只需要一系列“提示”(Prompt)即可，非常适合数学、代码等生成任务。这里我们使用经典的 GSM8K 数学推理数据集。详见后续章节。

## 第二部分：SFT 微调章节

**目标**：为模型注入基础知识或特定领域的说话风格。SFT 是最基础也是最重要的一步，它为后续更高级的对齐技术（如 DPO）打下坚实的基础。

**示例**：在本章节中，我们将使用在第一部分准备好的“甄嬛传”风格数据集 (`huanhuan_sft_local`)，通过 SFT-LoRA 技术，训练一个具有特定说话风格的模型。

#### 方案一：使用 LLaMA-Factory 进行 SFT

LLaMA-Factory 提供了高度集成的命令行工具，是进行 SFT 微调的最高效选择。

##### SFT 训练命令
下面是用于启动 SFT 训练的 bash 脚本。

```bash
#!/bin/bash
export CUDA_VISIBLE_DEVICES=0
export FORCE_TORCHRUN=1

MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
OUTPUT_PATH_SFT='./output/Zhenhuan_Style_SFT_LLF'
DATASET_NAME_SFT='huanhuan_sft_local'

llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --dataset $DATASET_NAME_SFT \
    --template qwen \
    --finetuning_type lora \
    --output_dir $OUTPUT_PATH_SFT \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 4096 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --plot_loss \
    --fp16 \
    \
    # --- QLoRA 量化参数 ---
    --quantization_bit 4 \
    --double_quantization True \
    --quantization_type nf4 \
    \
    # --- LoRA 参数 ---
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target q_proj,v_proj
```

##### 参数深度解析

这里对上述命令中的关键参数进行详细说明：

| 分类           | 参数                              | 说明                                                         |
| -------------- | --------------------------------- | ------------------------------------------------------------ |
| **核心参数**   | `--stage sft`                     | 指定当前任务为 **监督微调 (Supervised Fine-Tuning)** 阶段。  |
|                | `--do_train`                      | 明确指示脚本执行训练流程。                                   |
|                | `--model_name_or_path`            | 指定基础模型。可以是 Hugging Face Hub ID、ModelScope ID 或本地路径。 |
|                | `--dataset`                       | 指定在 `dataset_info.json` 中注册的数据集名称。              |
|                | `--template qwen`                 | **极其重要**。指定对话模板，必须与基础模型（如此处的 Qwen）严格匹配，否则模型无法正确理解输入。 |
|                | `--finetuning_type lora`          | 指定微调方法为 LoRA，这是一种高效的参数微调技术。            |
|                | `--output_dir`                    | 指定所有训练产物（模型权重、日志、检查点）的保存目录。       |
| **训练控制**   | `--overwrite_cache`               | 覆盖预处理后的数据缓存。当您修改了数据集或数据处理方式时，建议开启。 |
|                | `--overwrite_output_dir`          | 允许覆盖输出目录中已有的内容，方便重复实验。                 |
|                | `--cutoff_len 4096`               | 设置模型处理的最大序列长度（tokens）。需要根据您的任务和显存大小进行调整。 |
|                | `--per_device_train_batch_size 4` | 每块 GPU 在单次前向传播中处理的样本数量。                    |
|                | `--gradient_accumulation_steps 4` | 梯度累积步数。`有效批次大小 = batch_size * 累积步数`。这是在显存有限时扩大批次大小的常用技巧。 |
|                | `--learning_rate 1e-4`            | 学习率。对于 LoRA 微调，`1e-4` 是一个常用的、效果不错的初始值。 |
|                | `--num_train_epochs 3`            | 训练的总轮数。对于 SFT，通常 1-3 轮即可获得不错的效果。      |
|                | `--plot_loss`                     | 在训练结束后，在输出目录中生成一张训练损失曲线图 `training_loss.png`。 |
|                | `--fp16`                          | 启用半精度（16-bit）浮点数进行训练，可以大幅节省显存并提升训练速度。 |
| **QLoRA 量化** | `--quantization_bit 4`            | **QLoRA 核心**。指定使用 4-bit 对模型基座进行量化，极大降低了显存占用。 |
|                | `--double_quantization True`      | 启用双重量化，可以进一步节省少量显存，推荐开启。             |
|                | `--quantization_type nf4`         | 指定量化类型为 `nf4` (Normal Float 4)，这是 QLoRA 推荐的、理论上更优的 4-bit 数据类型。 |
| **LoRA 参数**  | `--lora_rank 8`                   | LoRA 矩阵的秩 (r)。决定了 LoRA 适配器的参数量大小。常用值为 8, 16, 32, 64。值越大，可训练参数越多，拟合能力越强，但过大也可能导致过拟合。 |
|                | `--lora_alpha 16`                 | LoRA 的缩放因子。通常设置为 `lora_rank` 的 2 倍，这是一个经验性的最佳实践。 |
|                | `--lora_dropout 0.05`             | 在 LoRA 矩阵上应用的 Dropout 比率，用于防止过拟合。          |
|                | `--lora_target all`               | **推荐设置**。指定将 LoRA 应用到模型中的哪些模块。设置为 `all` 后，LLaMA-Factory 会自动识别所有可应用的线性层（如 `q_proj`, `v_proj` 等），省去手动指定的麻烦。 |



##### 执行训练

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_DISABLED"] = "true"
# os.environ["WANDB_MODE"] = "offline"  # 离线模式

# --- 请在此处修改你的路径和名称 ---
# 模型ID，可以是 Hugging Face Hub ID 或 ModelScope ID
MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
OUTPUT_PATH='./output/Zhenhuan_Style_SFT'
# 使用您在步骤2.1和2.3中准备好的本地数据集
DATASET_NAME='huanhuan_sft_local'
# ---------------------------------
DS_CONFIG_PATH='examples/deepspeed/ds_z3_config.json'

!FORCE_TORCHRUN=1 llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --dataset $DATASET_NAME_SFT \
    --template qwen \
    --finetuning_type lora \
    --output_dir $OUTPUT_PATH_SFT \
    --overwrite_cache \
    --overwrite_output_dir \
    --cutoff_len 4096 \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-4 \
    --num_train_epochs 3 \
    --plot_loss \
    --fp16 \
    \
    --quantization_bit 4 \
    --double_quantization True \
    --quantization_type nf4 \
    \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target all \
    --report_to none

#### 方案二：使用 Hugging Face TRL 进行 SFT

对于希望更深入定制训练逻辑的用户，可以直接使用 Hugging Face 的 TRL 库。

In [ ]:
# train_sft_with_trl.py
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import SFTTrainer

def train_sft():
    # --- 1. 定义模型、数据集和输出路径 ---
    model_id = 'qwen/Qwen1.5-0.5B-Chat'
    dataset_name = "huanhuan_sft_local"
    output_dir = "./output/Zhenhuan_Style_SFT_TRL"

    # --- 2. 配置 QLoRA 量化 ---
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,                      # 激活4-bit量化
        bnb_4bit_quant_type="nf4",              # 量化类型 (nf4, fp4)
        bnb_4bit_compute_dtype=torch.bfloat16,  # 计算数据类型
        bnb_4bit_use_double_quant=True,         # 激活嵌套量化
    )

    # --- 3. 加载模型和分词器 ---
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map={"": 0} # 指定GPU
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    tokenizer.pad_token = tokenizer.eos_token

    # --- 4. 加载数据集 ---
    dataset = load_dataset("json", data_files=f"data/{dataset_name}.json", split="train")

    # --- 5. 配置训练参数 ---
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        learning_rate=1e-4,
        num_train_epochs=3,
        logging_steps=10,
        fp16=True,
        save_strategy="epoch",
    )

    # --- 6. 配置 LoRA ---
    peft_config = LoraConfig(
        r=8,
        lora_alpha=16,
        lora_dropout=0.05,
        # 指定要应用 LoRA 的模块。这是一个关键参数！
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
    )

    # --- 7. 创建并启动 SFTTrainer ---
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=dataset,
        dataset_text_field="instruction", # 指定包含文本的列
        max_seq_length=1024,
        peft_config=peft_config,
    )

    trainer.train()
    print(f"SFT 训练完成，模型已保存至 {output_dir}")

if __name__ == "__main__":
    train_sft()

运行脚本 python train_sft_with_trl.py。

#### SFT 模型合并与测试

无论使用哪种方案，训练完成后得到的都只是一个轻量的 LoRA 适配器（adapter），而不是一个完整的模型。我们需要将这个适配器与原始的基础模型合并，才能得到一个可以独立部署和使用的模型。



##### 合并模型

```bash
#!/bin/bash
export CUDA_VISIBLE_DEVICES=0

BASE_MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
ADAPTER_PATH='./output/Zhenhuan_Style_SFT'
EXPORT_PATH='./output/Zhenhuan_SFT_Merged'

llamafactory-cli export \
    --model_name_or_path $BASE_MODEL_PATH \
    --adapter_name_or_path $ADAPTER_PATH \
    --template qwen \
    --finetuning_type lora \
    --export_dir $EXPORT_PATH \
    --export_size 2 \
    --export_legacy_format False
```

In [ ]:
BASE_MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
ADAPTER_PATH='./output/Zhenhuan_Style_SFT'
EXPORT_PATH='./output/Zhenhuan_SFT_Merged'

!FORCE_TORCHRUN=1 llamafactory-cli export \
    --model_name_or_path $BASE_MODEL_PATH \
    --adapter_name_or_path $ADAPTER_PATH \
    --template qwen \
    --finetuning_type lora \
    --export_dir $EXPORT_PATH \
    --export_size 2 \
    --export_legacy_format False

##### 启动 Web UI 聊天测试

In [ ]:
#!/bin/bash
MERGED_MODEL_PATH='./output/Zhenhuan_SFT_Merged'

!llamafactory-cli webchat \
    --model_name_or_path $MERGED_MODEL_PATH \
    --template qwen

##### 通过代码进行交互式推理

通过 Python 脚本来调用模型，进行更灵活的测试。

In [ ]:
# -*- coding: utf-8 -*-
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# --- 请确保这里的路径指向您合并后的模型 ---
model_path = './output/Zhenhuan_SFT_Merged'

print(f"正在从 '{model_path}' 加载模型和分词器...")

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16, # 推荐使用 bfloat16 以获得更好的性能和兼容性
    device_map="auto" # 自动将模型加载到可用的 GPU
)
model.eval()

print("模型加载完成，准备生成回答。")

# --- 关键步骤：构建符合模板的对话 ---

# 1. 将你的问题构造成一个列表，其中包含字典，每个字典代表一个角色和其内容
messages = [
    {"role": "user", "content": "你好，可以介绍一下你是谁吗？"}
]

# 2. 使用 tokenizer.apply_chat_template 来格式化输入
#    这个函数会自动添加 <|im_start|>user\n...<|im_end|>\n<|im_start|>assistant\n 等特殊标记
#    add_generation_prompt=True 会在最后加上 assistant 的起始标记，引导模型开始生成
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# --- 生成回答 ---
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

# --- 解码并打印 ---
# outputs[0] 包含了输入的 prompt 和新生成的回答
# 我们需要从 outputs 中剥离掉输入的 prompt 部分，只解码新生成的内容
response_ids = outputs[0][inputs.shape[-1]:]
response = tokenizer.decode(response_ids, skip_special_tokens=True)

print("\n--- 对话 ---")
print("User:", messages[0]["content"])
print("Assistant:", response)

## 第三部分：DPO 微调章节

**目标**：在模型具备基础能力后，通过“好/坏”回答对比，让其学习人类的偏好，生成更优质的回答。



#### (可选) 预备步骤：SFT

**为什么需要 SFT？** 直接对基础模型进行 DPO 也可以，但通常效果不如先进行 SFT。SFT 阶段可以让模型先学习目标领域的基本知识和数据分布（例如，先学会如何回答知乎风格的问题），为后续的偏好学习提供一个更好的起点，从而让 DPO 训练更稳定、效果更好。

**示例**：使用我们从知乎偏好数据中提取的 SFT 数据集，先进行一轮 SFT。
```bash
#!/bin/bash
export CUDA_VISIBLE_DEVICES=0
export FORCE_TORCHRUN=1

MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
OUTPUT_PATH_SFT='./output/ZhihuQA_SFT'
DATASET_NAME_SFT='zhihu_sft_local'

llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --dataset $DATASET_NAME_SFT \
    --template qwen \
    --finetuning_type lora \
    --output_dir $OUTPUT_PATH_SFT \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-5 \
    --num_train_epochs 1 \
    --plot_loss \
    --fp16
    \
    # --- QLoRA 量化参数 ---
    --quantization_bit 4 \
    --double_quantization True \
    --quantization_type nf4 \
    \
    # --- LoRA 参数 ---
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target all \
    --report_to none


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["WANDB_DISABLED"] = "true"
# os.environ["WANDB_MODE"] = "offline"  # 离线模式
os.environ["FORCE_TORCHRUN"] = "1"

MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
OUTPUT_PATH_SFT='./output/ZhihuQA_SFT'
DATASET_NAME_SFT='zhihu_sft_local'

!llamafactory-cli train \
    --stage sft \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --dataset $DATASET_NAME_SFT \
    --template qwen \
    --finetuning_type lora \
    --output_dir $OUTPUT_PATH_SFT \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-5 \
    --num_train_epochs 1 \
    --plot_loss \
    --fp16 \
    --quantization_bit 4 \
    --double_quantization True \
    --quantization_type nf4 \
    --lora_rank 8 \
    --lora_alpha 16 \
    --lora_dropout 0.05 \
    --lora_target all

#### DPO 训练

现在，我们在 SFT 预训练过的模型基础上，使用完整的 DPO 数据集进行偏好对齐。

```bash
#!/bin/bash
export CUDA_VISIBLE_DEVICES=0
export FORCE_TORCHRUN=1
# 消除日志中的并行化警告，使输出更干净
export TOKENIZERS_PARALLELISM=false

# DPO时，--model_name_or_path 应指向原始基础模型
MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
# 使用 --adapter_name_or_path 加载 SFT 阶段的 LoRA 权重
SFT_ADAPTER_PATH='./output/ZhihuQA_SFT'
OUTPUT_PATH_DPO='./output/ZhihuQA_DPO'
DATASET_NAME_DPO='zhihu_dpo_local'

llamafactory-cli train \
    --stage dpo \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --adapter_name_or_path $SFT_ADAPTER_PATH \
    --dataset $DATASET_NAME_DPO \
    --template qwen \
    --finetuning_type lora \
    --lora_target all \
    --output_dir $OUTPUT_PATH_DPO \
    --per_device_train_batch_size 2 \
    --gradient_accumulation_steps 8 \
    --learning_rate 1e-6 \
    --num_train_epochs 1.0 \
    --plot_loss \
    --fp16 \
    --quantization_bit 4
```

In [ ]:
# DPO时，--model_name_or_path 应指向原始基础模型
MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
# 使用 --adapter_name_or_path 加载 SFT 阶段的 LoRA 权重
SFT_ADAPTER_PATH='./output/ZhihuQA_SFT'
OUTPUT_PATH_DPO='./output/ZhihuQA_DPO'
DATASET_NAME_DPO='zhihu_dpo_local'

!FORCE_TORCHRUN=1 llamafactory-cli train \
    --stage dpo \
    --do_train \
    --model_name_or_path $MODEL_PATH \
    --adapter_name_or_path $SFT_ADAPTER_PATH \
    --dataset $DATASET_NAME_DPO \
    --template qwen \
    --finetuning_type lora \
    --lora_target all \
    --output_dir $OUTPUT_PATH_DPO \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --learning_rate 1e-6 \
    --num_train_epochs 1.0 \
    --fp16 \
    --quantization_bit 4 \
    --gradient_checkpointing

#### Hugging Face TRL

In [ ]:
# train_dpo_with_trl.py
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from peft import LoraConfig
from trl import DPOTrainer

def train_dpo():
    # --- 1. 定义模型、数据集和输出路径 ---
    sft_model_id = "./output/ZhihuQA_SFT"
    dataset_name = "zhihu_dpo_local"
    output_dir = "./output/ZhihuQA_DPO_TRL"

    # --- 2. 配置 QLoRA 量化 ---
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_use_double_quant=True,
    )

    # --- 3. 加载模型和分词器 ---
    model = AutoModelForCausalLM.from_pretrained(sft_model_id, quantization_config=bnb_config, device_map={"": 0})
    tokenizer = AutoTokenizer.from_pretrained(sft_model_id)
    tokenizer.pad_token = tokenizer.eos_token

    # --- 4. 加载数据集 ---
    dataset = load_dataset("json", data_files=f"data/{dataset_name}.json", split="train")

    # --- 5. 配置训练参数 ---
    training_args = TrainingArguments(
        output_dir=output_dir,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        learning_rate=1e-6,
        num_train_epochs=1,
        logging_steps=5,
        fp16=True,
        save_strategy="epoch",
    )

    # --- 6. 配置 LoRA ---
    peft_config = LoraConfig(r=8, lora_alpha=16, lora_dropout=0.05, target_modules=["q_proj", "v_proj"], task_type="CAUSAL_LM")

    # --- 7. 创建并启动 DPOTrainer ---
    dpo_trainer = DPOTrainer(
        model,
        args=training_args,
        beta=0.1,
        train_dataset=dataset,
        tokenizer=tokenizer,
        peft_config=peft_config,
        dataset_map_kwargs={"prompt": "prompt", "chosen": "chosen", "rejected": "rejected"}
    )

    dpo_trainer.train()
    print(f"DPO 训练完成，模型已保存至 {output_dir}")

if __name__ == "__main__":
    train_dpo()


运行脚本 python train_dpo_with_trl.py。

### DPO 模型合并与测试

- **合并模型**

```
#!/bin/bash
export CUDA_VISIBLE_DEVICES=0

BASE_MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
ADAPTER_PATH='./output/ZhihuQA_DPO'
EXPORT_PATH='./output/ZhihuQA_DPO_Merged'

llamafactory-cli export \
    --model_name_or_path $BASE_MODEL_PATH \
    --adapter_name_or_path $ADAPTER_PATH \
    --template qwen \
    --finetuning_type lora \
    --export_dir $EXPORT_PATH \
    --export_size 2 \
    --export_legacy_format False
```



In [ ]:
BASE_MODEL_PATH='qwen/Qwen1.5-0.5B-Chat'
ADAPTER_PATH='./output/ZhihuQA_DPO'
EXPORT_PATH='./output/ZhihuQA_DPO_Merged'

!llamafactory-cli export \
    --model_name_or_path $BASE_MODEL_PATH \
    --adapter_name_or_path $ADAPTER_PATH \
    --template qwen \
    --finetuning_type lora \
    --export_dir $EXPORT_PATH \
    --export_size 2 \
    --export_legacy_format False

- **启动 Web UI 聊天测试**

```
#!/bin/bash
MERGED_MODEL_PATH='./output/ZhihuQA_DPO_Merged'

llamafactory-cli webchat \
    --model_name_or_path $MERGED_MODEL_PATH \
    --template qwen
```

- 代码推理

如果你无法启动 Web UI，或者希望在代码中直接调用模型，可以使用以下 Python 脚本进行交互式推理。

In [ ]:
# inference.py
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# --- 请确保这里的路径指向您合并后的DPO模型 ---
model_path = './output/ZhihuQA_DPO_Merged'

print(f"正在从 '{model_path}' 加载模型和分词器...")

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
model.eval()

print("模型加载完成，准备生成回答。")

# 构造对话
messages = [
    {"role": "user", "content": "请问，为什么天空是蓝色的？"}
]

# 使用 apply_chat_template 格式化输入
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

# 生成回答
with torch.no_grad():
    outputs = model.generate(
        inputs,
        max_new_tokens=512,
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

# 解码并打印
response_ids = outputs[0][inputs.shape[-1]:]
response = tokenizer.decode(response_ids, skip_special_tokens=True)

print("\n--- 对话 ---")
print("User:", messages[0]["content"])
print("Assistant:", response)

## 第四部分：DeepSpeed 与多卡训练配置

### DeepSeed配置

**DeepSpeed 配置**: LLaMA-Factory 在 `examples/deepspeed` 目录下提供了多种配置文件，如 `ds_z3_config.json` (ZeRO Stage 3)。这些配置通过优化内存使用，使得在有限的硬件上训练更大的模型成为可能。

**多卡/多机训练**:

- **单机多卡**: 只需修改 `CUDA_VISIBLE_DEVICES` 环境变量，例如 `export CUDA_VISIBLE_DEVICES=0,1`，然后使用 `torchrun` 启动训练脚本。
- **多机多卡**: 需要创建一个 `hostfile` 文件，列出所有机器的 IP 地址和 GPU 数量，并在 `torchrun` 命令中指定该文件。所有机器需要能够免密 SSH 访问。

#### 准备 DeepSpeed 配置文件

DeepSpeed 官方仓库提供示例文件：
- GitHub：https://github.com/microsoft/DeepSpeed
- 常见位置：examples 文件夹里，例如 ds_config.json

LLaMA-Factory 官方文档也有推荐配置：
- https://github.com/liuhaotian/LLaMA-Factory
- DeepSpeed 配置: LLaMA-Factory 在 examples/deepspeed 目录下提供了多种配置文件，如 ds_z3_config.json (ZeRO Stage 3)。这些配置通过优化内存使用，使得在有限的硬件上训练更大的模型成为可能。

##### DeepSpeed 单机单卡配置
```json
{
  "train_batch_size": 4,
  "train_micro_batch_size_per_gpu": 4,
  "gradient_accumulation_steps": 1,
  "steps_per_print": 50,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 5e-5,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 0.01
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 5e-5,
      "warmup_num_steps": 100
    }
  },
  "fp16": {
    "enabled": true,
    "loss_scale": 0
  },
  "zero_optimization": {
    "stage": 1,
    "offload_optimizer": {
      "device": "none"
    },
    "offload_param": {
      "device": "none"
    },
    "overlap_comm": false,
    "contiguous_gradients": true
  },
  "gradient_clipping": 1.0,
  "wall_clock_breakdown": false,
  "steps_per_checkpoint": 200,
  "activation_checkpointing": {
    "partition_activations": false,
    "contiguous_memory_optimization": true
  }
}
```

说明：适合本地单卡快速调试和小模型训练，不需要 offload，也不开激活分区。

##### DeepSpeed 单机多卡配置

```json
{
  "train_batch_size": 32,
  "train_micro_batch_size_per_gpu": 4,
  "gradient_accumulation_steps": 2,
  "steps_per_print": 50,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 5e-5,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 0.01
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 5e-5,
      "warmup_num_steps": 500
    }
  },
  "fp16": {
    "enabled": true,
    "loss_scale": 0
  },
  "zero_optimization": {
    "stage": 2,
    "offload_optimizer": {
      "device": "none"
    },
    "offload_param": {
      "device": "none"
    },
    "overlap_comm": true,
    "contiguous_gradients": true
  },
  "gradient_clipping": 1.0,
  "wall_clock_breakdown": false,
  "steps_per_checkpoint": 500,
  "activation_checkpointing": {
    "partition_activations": true,
    "contiguous_memory_optimization": true
  }
}

```

说明：适合单机多卡训练，开启 ZeRO stage 2，支持梯度累积和激活检查点节省显存。


##### DeepSpeed 多机多卡配置

```json
{
  "train_batch_size": 128,
  "train_micro_batch_size_per_gpu": 4,
  "gradient_accumulation_steps": 4,
  "steps_per_print": 50,
  "optimizer": {
    "type": "AdamW",
    "params": {
      "lr": 5e-5,
      "betas": [0.9, 0.999],
      "eps": 1e-8,
      "weight_decay": 0.01
    }
  },
  "scheduler": {
    "type": "WarmupLR",
    "params": {
      "warmup_min_lr": 0,
      "warmup_max_lr": 5e-5,
      "warmup_num_steps": 1000
    }
  },
  "fp16": {
    "enabled": true,
    "loss_scale": 0
  },
  "zero_optimization": {
    "stage": 3,
    "offload_optimizer": {
      "device": "cpu"
    },
    "offload_param": {
      "device": "cpu"
    },
    "overlap_comm": true,
    "contiguous_gradients": true
  },
  "gradient_clipping": 1.0,
  "wall_clock_breakdown": true,
  "steps_per_checkpoint": 1000,
  "activation_checkpointing": {
    "partition_activations": true,
    "contiguous_memory_optimization": true
  }
}

```

说明：适合多机分布式大模型训练，开启 ZeRO stage 3，参数和优化器 offload 到 CPU/NVMe，开启通信重叠和详细时间统计。


| 配置   | train\_batch\_size | micro\_batch | ZeRO stage | Offload | 激活检查点 | 单机单卡可用  |
| ---- | ------------------ | ------------ | ---------- | ------- | ----- | ------- |
| 单机单卡 | 4                  | 4            | 1          | none    | false | ✅       |
| 单机多卡 | 32                 | 4            | 2          | none    | true  | ✅       |
| 多机多卡 | 128                | 4            | 3          | cpu     | true  | ❌（多机必用） |


##### DeepSeed 配置参数

| 字段                                                        | 默认值          | 可选值             | 说明                  | 通用建议                         |
| --------------------------------------------------------- | ------------ | --------------- | ------------------- | ---------------------------- |
| `train_batch_size`                                        | 16           | 正整数             | 所有 GPU 累积 batch     | 按显存和卡数调节，单机单卡可以小一点           |
| `train_micro_batch_size_per_gpu`                          | 4            | 1\~16           | 每张 GPU 的 batch size | 显存够大可增加，推荐 1\~4              |
| `gradient_accumulation_steps`                             | 4            | 1\~16           | 梯度累积步数              | 用于增加有效 batch size            |
| `steps_per_print`                                         | 50           | 10\~100         | 日志打印频率              | 单机 10\~50 足够                 |
| `optimizer.type`                                          | AdamW        | Adam, SGD       | 优化器类型               | AdamW 通用微调                   |
| `optimizer.params.lr`                                     | 5e-5         | 1e-6\~5e-4      | 学习率                 | 小模型可略大，大模型保持 1e-5\~5e-5      |
| `optimizer.params.betas`                                  | \[0.9,0.999] | -               | AdamW 参数            | 通用默认                         |
| `optimizer.params.eps`                                    | 1e-8         | -               | AdamW 参数            | 通用默认                         |
| `optimizer.params.weight_decay`                           | 0.01         | 0\~0.1          | AdamW 参数            | 避免过拟合                        |
| `scheduler.type`                                          | WarmupLR     | Cosine, Linear  | 学习率调度               | WarmupLR 通用                  |
| `scheduler.params.warmup_min_lr`                          | 0            | ≥0              | 初始学习率               | 默认 0                         |
| `scheduler.params.warmup_max_lr`                          | 5e-5         | 0\~1e-3         | 最大学习率               | 按 lr 调整                      |
| `scheduler.params.warmup_num_steps`                       | 500          | 100\~2000       | Warmup 步数           | 小数据集 100~~500，大数据集 500~~2000 |
| `fp16.enabled`                                            | true         | true/false      | 是否开启混合精度            | 节省显存，加速训练                    |
| `fp16.loss_scale`                                         | 0            | 正整数             | 动态 loss scale       | 0 自动即可                       |
| `fp16.loss_scale_window`                                  | 1000         | -               | 调整窗口                | 通用默认                         |
| `fp16.hysteresis`                                         | 2            | -               | 动态 loss scale       | 通用默认                         |
| `fp16.min_loss_scale`                                     | 1            | -               | 最小 loss scale       | 通用默认                         |
| `zero_optimization.stage`                                 | 2            | 0\~3            | ZeRO 优化阶段           | 单机单卡可 1 或 2，多卡可 3            |
| `zero_optimization.offload_optimizer.device`              | none         | cpu, nvme, none | 优化器 offload         | 多机大模型必开                      |
| `zero_optimization.offload_param.device`                  | none         | cpu, nvme, none | 参数 offload          | 多机多卡可开                       |
| `zero_optimization.overlap_comm`                          | true         | true/false      | 通信和计算重叠             | 多机必开，单机可关                    |
| `zero_optimization.contiguous_gradients`                  | true         | true/false      | 梯度连续内存              | 通用保持 true                    |
| `gradient_clipping`                                       | 1.0          | ≥0              | 防止梯度爆炸              | 1.0 足够                       |
| `wall_clock_breakdown`                                    | false        | true/false      | 打印详细时间              | 单机 False 就够                  |
| `steps_per_checkpoint`                                    | 500          | 正整数             | 每多少步保存 checkpoint   | 数据量小可调小                      |
| `activation_checkpointing.partition_activations`          | true         | true/false      | 激活检查点               | 大模型必开                        |
| `activation_checkpointing.contiguous_memory_optimization` | true         | true/false      | 内存优化                | 通用保持 True                    |




单机单卡：只会在一张 GPU 上训练，micro batch 小，ZeRO stage 可降为 1。

单机多卡：train_micro_batch_size_per_gpu × GPU 数 = train_batch_size。开启 ZeRO stage 2 或 3，offload 可选 CPU/NVMe。

多机多卡：DeepSpeed 启动命令加 --num_nodes、--num_gpus，ZeRO stage 2~3，offload 参数必开。

### LLaMA-Factory + DeepSpeed 启动命令模板

DeepSpeed + PyTorch + LLaMA-Factory 都支持 TensorBoard 本地可视化：

```bash
pip install tensorboard
```

- 训练脚本中加参数：

```bash
--report_to tensorboard
```

然后启动 TensorBoard：

```bash
tensorboard --logdir ./output/Zhenhuan_Style_SFT
```


在浏览器中访问 http://localhost:6006 即可看到训练曲线、loss、lr 等指标。


##### 训练参数

###### 环境变量参数

| 参数/变量名            | 作用                   | 备注                                             |
| ---------------------- | ---------------------- | ------------------------------------------------ |
| `CUDA_VISIBLE_DEVICES` | 指定使用的 GPU 索引    | 从 0 开始。0 表示使用第一张卡。                  |
| `FORCE_TORCHRUN`       | 强制使用 torchrun 启动 | 解决 LLaMA-Factory 中 DeepSpeed 的启动检查问题。 |
| `MODEL_PATH`           | 基础模型的路径或 ID    | 可为 HuggingFace Hub、ModelScope ID 或本地路径。 |
| `OUTPUT_PATH_SFT`      | 训练输出目录           | 存放 checkpoints、适配器权重和日志。             |
| `DATASET_NAME_SFT`     | 使用的数据集名称       | 必须是 `dataset_info.json` 中注册过的名称。      |

------



###### 核心参数

| 参数                     | 作用             | 备注                                  |
| ------------------------ | ---------------- | ------------------------------------- |
| `--stage sft`            | 指定训练阶段     | `sft` 表示监督微调。                  |
| `--do_train`             | 执行训练         | 必须添加此参数才会开始训练。          |
| `--model_name_or_path`   | 模型名称或路径   | 引用 `$MODEL_PATH`。                  |
| `--dataset`              | 数据集名称或路径 | 引用 `$DATASET_NAME_SFT`。            |
| `--template qwen`        | 对话模板         | 确保与基础模型（如 Qwen）的格式一致。 |
| `--finetuning_type lora` | 微调类型         | `lora` 是最常用的高效微调方法。       |
| `--output_dir`           | 输出目录         | 引用 `$OUTPUT_PATH_SFT`。             |

------



###### 训练控制参数

| 参数                            | 作用             | 备注                                             |
| ------------------------------- | ---------------- | ------------------------------------------------ |
| `--overwrite_cache`             | 覆盖数据缓存     | 更换数据集或格式后建议使用。                     |
| `--overwrite_output_dir`        | 覆盖输出目录     | 允许覆盖之前训练产生的文件。                     |
| `--cutoff_len`                  | 最大序列长度     | 单位为 token，依据显存与任务设置。               |
| `--per_device_train_batch_size` | 单卡 batch size  | 越大通常训练更稳定，受限于显存。                 |
| `--gradient_accumulation_steps` | 梯度累积步数     | 提升等效 batch size，适用于显存有限情况。        |
| `--learning_rate`               | 学习率           | QLoRA 推荐起点为 `1e-4`。                        |
| `--num_train_epochs`            | 训练轮数         | SFT 阶段通常 1~3 轮即可。                        |
| `--plot_loss`                   | 绘制训练损失曲线 | 会生成 `training_loss.png`，保存于输出目录。     |
| `--fp16`                        | 启用半精度训练   | 使用 16-bit 浮点数训练，节省显存、提升训练速度。 |

------



###### QLoRA 量化参数

| 参数                      | 作用     | 备注                                         |
| ------------------------- | -------- | -------------------------------------------- |
| `--quantization_bit 4`    | 量化位数 | 4-bit 是 QLoRA 的核心设定。                  |
| `--double_quantization`   | 双重量化 | 节省显存，性能影响小，推荐开启。             |
| `--quantization_type nf4` | 量化类型 | `nf4 (Normal Float 4)` 是推荐的 4-bit 类型。 |

------



###### LoRA 参数

| 参数             | 作用              | 备注                                              |
| ---------------- | ----------------- | ------------------------------------------------- |
| `--lora_rank`    | LoRA 矩阵秩 (r)   | 决定可训练参数量，常用值：8, 16, 32, 64。         |
| `--lora_alpha`   | LoRA 缩放因子     | 通常设置为 `lora_rank` 的 2 倍。                  |
| `--lora_dropout` | LoRA Dropout 比例 | 常设为 0.05 或 0.1，用于防止过拟合。              |
| `--lora_target`  | LoRA 目标层       | **推荐做法**：对于 LLaMA-Factory，可以直接设置为 all，框架会自动识别并选择模型中所有可用的线性层（如 q_proj, v_proj, k_proj, o_proj, gate_proj 等），这是最省心且效果通常最好的选择。 |

单机单卡可以直接运行上面的脚本，DeepSpeed 会自动管理内存和优化。

如果以后想多机，只需在命令里加上：

```bash
--hostfile ./my_hostfile
```

并在 hostfile 中列出各机器 IP 和 GPU 数量，DeepSpeed 会自动分布式训练。


##### hostfile.txt 示例（单机单卡）

```
localhost slots=1
```

- **localhost**：本机名称或 IP 地址
- **slots=1**：本机可用 GPU 数量，这里是 1 张 GPU

如果你有多张 GPU，可以写成：

```
localhost slots=4
```

表示本机有 4 张 GPU 可用，DeepSpeed 会在 4 张卡上分布训练任务。



##### hostfile.txt 示例（多机多卡）

假设有两台机器：

- 机器 A，IP: 192.168.1.10，有 4 张 GPU
- 机器 B，IP: 192.168.1.11，有 2 张 GPU

hostfile 可以写成：

```
192.168.1.10 slots=4
192.168.1.11 slots=2
```

- DeepSpeed 会自动把 batch 和计算任务按 GPU 数量分配
- 只需在 **一台机器上运行训练命令**，DeepSpeed 会通过 SSH 调用其他机器

##### 注意事项

1. **单机单卡**：hostfile 可以省略，直接在本机跑即可
2. **多机训练**：
   - 确保所有机器能通过 SSH 免密访问
   - 所有机器都安装好相同版本的 Python、依赖和 PyTorch/DeepSpeed
3. **slots 和 batch size**：每张 GPU batch size * slots = 全局 batch size
4. **单机多卡**：hostfile 也可以写 localhost slots=4 这样 DeepSpeed 会按 GPU 分配

## 第五部分：GRPO 微调




### GRPO (Generative Replay Policy Optimization)

**目标**：提升模型的推理能力和事实准确性。GRPO 是一种新颖的对齐算法，它通过让模型重新生成并评估自己的答案来进行学习和优化，特别适合数学计算、代码生成、事实问答等需要高精度的任务。

**本章方案**：我们将采用 `TRL` 和 `EasyR1` 的原生工作流。首先在本章内准备 `GSM8K` 数据文件，然后使用 `TRL` 库编写脚本完成 SFT 预备训练，最后切换到 `EasyR1` 框架来执行专业的 GRPO 训练。

### GSM8K 数据集准备

在开始训练之前，我们需要为 GRPO 准备 `GSM8K` 数据集文件。

In [ ]:
# file: prepare_gsm8k_for_grpo.py
import json
import os
from datasets import load_dataset
from tqdm import tqdm

# --- 1. 配置与辅助函数 ---

# 定义系统提示，这是我们希望模型遵循的指令，确保输出格式统一
SYSTEM_PROMPT = """
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

# 定义一个“思维链”(Chain-of-Thought)示例，用于给模型一个清晰的模仿样本
XML_COT_FORMAT = """\
<reasoning>
The single-digit numbers are 1, 2, 3, 4, 5, 6, 7, 8, 9. A prime number is a number greater than 1 that has no positive divisors other than 1 and itself. 7 is a prime number. 9 is divisible by 3. 8 is divisible by 2.
</reasoning>
<answer>
7
</answer>
"""

def ensure_dir_exists(path):
    """确保文件所在的目录存在。"""
    dir_name = os.path.dirname(path)
    if dir_name and not os.path.exists(dir_name):
        os.makedirs(dir_name)
        print(f"创建目录: {dir_name}")

def extract_hash_answer(text: str) -> str | None:
    """
    从原始答案文本（例如 '...过程... #### 7'）中稳健地提取最终的纯数字答案。
    """
    if "####" not in text:
        return None
    # 分割字符串并取最后一部分，去除可能的前后空格
    return text.split("####")[1].strip()

# --- 2. 核心格式化函数 ---

def format_gsm8k_for_grpo(dataset, output_path, use_one_shot=True):
    """
    加载原始 GSM8K 数据集，将其完全格式化为 GRPO 训练所需的最终格式，并保存到 JSONL 文件。

    最终格式: {"prompt": [{"role": "system", ...}, {"role": "user", ...}], "answer": "..."}
    """
    ensure_dir_exists(output_path)

    print(f"开始进行完整的格式化处理并保存到 {output_path}...")

    processed_count = 0
    with open(output_path, 'w', encoding='utf-8') as f:
        # 使用 tqdm 显示进度条
        for item in tqdm(dataset, desc="格式化 GSM8K 数据"):
            # 1. 构建聊天格式的 prompt 列表，首先是系统提示
            prompt_list = [{'role': 'system', 'content': SYSTEM_PROMPT}]

            # 2. (可选) 添加 one-shot 示例来引导模型学习格式
            if use_one_shot:
                prompt_list.extend([
                    {'role': 'user', 'content': 'What is the largest single-digit prime number?'},
                    {'role': 'assistant', 'content': XML_COT_FORMAT}
                ])

            # 3. 添加当前样本的实际问题 (原始键名为 'question')
            prompt_list.append({'role': 'user', 'content': item['question']})

            # 4. 提取纯净答案 (原始键名为 'answer')
            answer_text = item.get('answer', '')
            extracted_answer = extract_hash_answer(answer_text)

            # 5. 只写入包含有效答案的样本，确保数据质量
            if extracted_answer is not None:
                formatted_item = {
                    "prompt": prompt_list,
                    "answer": extracted_answer
                }
                f.write(json.dumps(formatted_item, ensure_ascii=False) + '\n')
                processed_count += 1

    print(f"✅ 处理完成！共 {processed_count} 条有效样本已保存到: {output_path}")


print("正在从 Hugging Face Hub 加载 gsm8k 数据集...")
gsm8k_dataset = load_dataset("gsm8k", "main", split="train")
format_gsm8k_for_grpo(gsm8k_dataset, "data/gsm8k_grpo_local.jsonl")


运行 python prepare_gsm8k_for_grpo.py 来生成 GRPO 训练所需的数据文件。

### 使用 EasyR1 执行 GRPO 训练

现在，我们使用 `EasyR1` 框架在 SFT 的基础上进行 GRPO 训练。

#### 安装 EasyR1

In [ ]:
os.chdir(work_dir)
print(f"📂 当前工作目录已切换到: {work_dir.resolve()}")

In [ ]:

# # 克隆 EasyR1 仓库
# !git clone https://github.com/hiyouga/EasyR1.git

# # 安装 EasyR1 及其依赖
# %cd EasyR1
# !pip install -e .

In [ ]:
# MODEL_PATH="Qwen/Qwen1.5-0.5B-Chat"

# !python -m verl.trainer.main \
#     config=examples/config.yaml \
#     worker.actor.model.model_path=${MODEL_PATH} \
#     dataset.train_path=data/gsm8k_grpo_local.jsonl


### 执行 GRPO 训练 (使用 TRL 自定义脚本)

创建一个完整的 Python 脚本 `train_grpo.py`，它将实现 GRPO 的训练循环。

In [ ]:
# file: train_grpo.py
import re
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig
from trl import GRPOConfig, GRPOTrainer
import os
import logging

# --- 1. 基本设置和日志记录 ---
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# --- 新增：环境诊断检查 ---
if torch.cuda.is_available():
    logger.info(f"✅ CUDA is available. Found {torch.cuda.device_count()} GPU(s).")
    logger.info(f"Current device: {torch.cuda.get_device_name(0)}")
else:
    logger.warning("⚠️ CUDA is not available. Training will run on CPU. Please check your NVIDIA driver and PyTorch installation.")

# --- 2. 通过环境变量进行配置 ---
MODEL_NAME = os.getenv("MODEL_NAME", "qwen/Qwen1.5-0.5B-Chat")
# 指向由 prepare_gsm8k_for_grpo.py 生成的、已完全格式化好的数据集
DATASET_PATH = os.getenv("DATASET_PATH", "data/gsm8k_grpo_local.jsonl")
OUTPUT_DIR = os.getenv("OUTPUT_DIR", "outputs/GSM8K_GRPO_Final")
RUN_NAME = os.getenv("RUN_NAME", "grpo-qwen1.5-0.5b-gsm8k-final")

os.makedirs(OUTPUT_DIR, exist_ok=True)
logger.info(f"模型 ID: {MODEL_NAME}")
logger.info(f"数据集路径: {DATASET_PATH}")
logger.info(f"输出目录: {OUTPUT_DIR}")
logger.info(f"运行名称: {RUN_NAME}")

# --- 3. 数据集加载 ---
# 数据已由预处理脚本完全格式化，此处直接加载即可。
try:
    logger.info(f"正在从预处理文件加载数据集: {DATASET_PATH}")
    dataset = load_dataset('json', data_files=DATASET_PATH, split='train')
    logger.info(f"数据集加载成功，样本数量: {len(dataset)}")
except Exception as e:
    logger.error(f"加载预处理数据集失败: {e}")
    raise

# --- 4. 奖励函数定义 ---
def extract_xml_answer(text: str) -> str:
    """从模型生成的 XML 格式文本中提取 <answer> 标签内的内容。"""
    try:
        answer = text.split("<answer>")[-1].split("</answer>")[0].strip()
        return answer
    except IndexError:
        logger.warning("Failed to extract answer from XML format.")
        return ""

def correctness_reward_func(prompts, completions, answer, **kwargs) -> list[float]:
    """主要奖励：根据答案正确性给予高分。"""
    responses = [comp[0]['content'] for comp in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    if prompts and answer and responses and extracted_responses:
        logger.info(f"问题:\n{prompts[0][-1]['content']}\n标准答案:\n{answer[0]}\n模型输出:\n{responses[0]}\n提取的答案:\n{extracted_responses[0]}")
    return [2.0 if resp == ans else 0.0 for resp, ans in zip(extracted_responses, answer)]

def int_reward_func(completions, **kwargs) -> list[float]:
    """辅助奖励：如果答案是数字，给予少量分数。"""
    responses = [comp[0]['content'] for comp in completions]
    extracted_responses = [extract_xml_answer(r) for r in responses]
    return [0.5 if r.isdigit() else 0.0 for r in extracted_responses]

def format_reward_func(completions, **kwargs) -> list[float]:
    """辅助奖励：如果格式基本正确，给予少量分数。"""
    pattern = r"<reasoning>.*?</reasoning>\s*<answer>.*?</answer>"
    responses = [comp[0]["content"] for comp in completions]
    return [0.5 if re.search(pattern, r, re.DOTALL) else 0.0 for r in responses]

def xmlcount_reward_func(completions, **kwargs) -> list[float]:
    """辅助奖励：根据 XML 标签的完整性给予精细分数。"""
    def count_xml(text):
        score = 0.0
        if "<reasoning>" in text: score += 0.125
        if "</reasoning>" in text: score += 0.125
        if "<answer>" in text: score += 0.125
        if "</answer>" in text: score += 0.125
        return score
    contents = [comp[0]["content"] for comp in completions]
    return [count_xml(c) for c in contents]

# --- 5. 模型和 Tokenizer 加载 ---
try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.bfloat16,
        device_map="auto" # 自动将模型加载到 GPU
    )
    logger.info("模型加载成功。")

    # --- （可选）高级优化 ---
    # 如果你使用的是 PyTorch 2.0 或更高版本，取消下面的注释可以极大地加速训练
    # model = torch.compile(model)
    # print("INFO: 已启用 torch.compile() 进行加速。")
except Exception as e:
    logger.error(f"加载模型失败: {e}.")
    raise

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# --- 6. PEFT (LoRA) 配置 ---
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules="all-linear", # 自动匹配所有线性层，避免出错
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
)

# --- 7. 训练参数配置 ---
training_args = GRPOConfig(
    output_dir=OUTPUT_DIR,
    run_name=RUN_NAME,
    learning_rate=5e-6,
    logging_steps=5,
    # bf16=True,
    fp16=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    num_generations=4,
    max_prompt_length=512,
    max_completion_length=256,
    num_train_epochs=1,
    save_steps=10,
    report_to="none",
    remove_unused_columns=False,
)

# --- 8. 初始化 GRPOTrainer ---
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[
        xmlcount_reward_func,
        format_reward_func,
        int_reward_func,
        correctness_reward_func
    ],
    args=training_args,
    train_dataset=dataset,
    processing_class=tokenizer,
    peft_config=peft_config
)

# --- 9. 开始训练 ---
logger.info("开始 GRPO 训练...")
try:
    trainer.train()
    logger.info("训练完成！")
except Exception as e:
    logger.error(f"训练过程中发生错误: {e}", exc_info=True)
    raise

### 推理测试


In [ ]:
# file: inference.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import os
import re
import glob

# --- 1. 配置 ---
# 基础模型 ID (必须与训练时使用的模型一致)
BASE_MODEL_NAME = os.getenv("MODEL_NAME", "qwen/Qwen1.5-0.5B-Chat")
# 训练时使用的输出目录 (与你的训练脚本保持一致)
OUTPUT_DIR_FROM_TRAINING = os.getenv("OUTPUT_DIR", "outputs/GSM8K_GRPO_Final")

# --- 自动查找最新的 checkpoint ---
print(f"INFO: 正在 '{OUTPUT_DIR_FROM_TRAINING}' 目录中查找最新的 checkpoint...")
checkpoints = glob.glob(os.path.join(OUTPUT_DIR_FROM_TRAINING, "checkpoint-*"))
if not checkpoints:
    raise ValueError(f"在目录 '{OUTPUT_DIR_FROM_TRAINING}' 中找不到任何 checkpoint。请确保路径正确且训练已保存。")

# 按数字大小排序找到最新的 checkpoint
latest_checkpoint = max(checkpoints, key=lambda x: int(x.split('-')[-1]))
ADAPTER_PATH = latest_checkpoint
print(f"INFO: 自动找到最新的 checkpoint: {ADAPTER_PATH}")


# --- 2. 加载模型和适配器 ---
print(f"INFO: 正在加载基础模型: {BASE_MODEL_NAME}...")
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_NAME,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)
print("INFO: 基础模型加载成功。")

print(f"INFO: 正在加载 Tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

print(f"INFO: 正在加载 LoRA 适配器: {ADAPTER_PATH}...")
# 加载 LoRA 适配器并将其应用到基础模型上
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH)
print("INFO: LoRA 适配器加载成功。")

# --- 3. 合并模型 ---
print("INFO: 正在合并模型和适配器...")
# 将 LoRA 模块的权重合并到基础模型中
model = model.merge_and_unload()
print("INFO: 模型合并完成。")

# --- 4. 推理函数 ---
# 定义与训练时一致的系统提示
SYSTEM_PROMPT = """
Respond in the following format:

<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""

def generate_response(question: str):
    """
    使用微调后的模型生成对给定问题的回答。
    """
    # 1. 构建聊天格式的 prompt
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

    # 2. 应用聊天模板并转换为输入张量
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # 3. 使用模型生成回答
    print("\n" + "="*50)
    print(f"问题: {question}")
    print("模型正在生成回答...")

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=256, # 与训练时的 max_completion_length 保持一致
        do_sample=False # 使用贪心解码以获得确定性输出
    )

    # 4. 解码并清理输出
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    print("--- 模型输出 ---")
    print(response)
    print("="*50 + "\n")

# --- 5. 执行推理 ---
if __name__ == "__main__":
    # 准备一些测试问题
    test_questions = [
        "Natalia sold 48/2 = 24 clips in the morning. Then she sold 12 clips in the afternoon. How many clips did Natalia sell in total?",
        "There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done, there will be 21 trees. How many trees did the grove workers plant today?",
        "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?",
    ]

    for q in test_questions:
        generate_response(q)


## 第六部分：GRPO 从入门到实践完整教程

## 前言

本指南将带你走过一条清晰的学习路径，从宏观概念到理论深度，最终落于代码实践。学习路径如下：

1. **核心思想与定位**：快速建立对 GRPO 的直观认知，明白它是什么，以及它在众多对齐算法中的位置。
2. **深入原理与推导**：理解 GRPO 的数学内核，学习其损失函数如何从第一性原理构建。
3. **动手实践与代码**：通过示例，实现自定义的 `GRPOTrainer`。

------

## 核心思想与定位

**一句话理解 GRPO**：**“擒贼先擒王”**

想象老师教学生写作文：给出一个标准答案（chosen）和多个错误范例（rejected），目标是生成标准答案，同时避开最容易混淆的错误。GRPO 的精髓是：

- **关注最难的负样本**：在一堆 rejected 样本里，模型只需战胜最强的那个。
- **整体考虑偏好群组**：与 DPO 只处理一对一不同，GRPO 一次性考虑所有 rejected 样本。

### GRPO 的诞生动机

在现实标注中，一个 prompt 往往对应一个好回答和多个坏回答。GRPO 的目标是**更高效地利用“一对多”偏好数据**，把学习重点放在最难区分的负样本上。

### 横向对比

| 特性     | PPO                     | DPO                        | GRPO                                     |
| -------- | ----------------------- | -------------------------- | ---------------------------------------- |
| 核心目标 | 最大化累积奖励          | 直接从偏好对学习           | 从偏好群组学习，处理“一对多”             |
| 数据类型 | (state, action, reward) | (prompt, chosen, rejected) | (prompt, chosen, [rejected_1,...])       |
| 工作方式 | 依赖奖励模型            | 跳过奖励模型，用逻辑损失   | DPO 的扩展，用 Hinge Loss 专注最强负样本 |
| 对齐方法 | RLHF 核心算法           | 直接对齐方法，不需奖励模型 | DPO 变体，不需奖励模型                   |

## GRPO 深入原理与推导

### 起点：DPO 的基石

要理解 GRPO，必须先理解 DPO 的逻辑。

#### 偏好量化（Bradley-Terry 模型）

我们如何将“人类更喜欢 $A$ 而不是 $B$”这个模糊概念数学化？Bradley-Terry 模型提供了一个优雅的方案。它假设每个选项背后都有一个潜在的“奖励分数” $r^*$，而偏好概率取决于这些分数的差异：

$$
p^*(y_c \succ y_r \mid x) = \sigma\big(r^*(x, y_c) - r^*(x, y_r)\big)
$$

这个公式是所有推导的基石，它将主观偏好转化为了一个可计算的概率。

#### 奖励定义（DPO 核心洞察）

传统方法需要训练一个独立的奖励模型来估算 $r^*$。DPO 的天才之处在于，它发现奖励可以直接用策略模型 $\pi_\theta$ 和参考模型 $\pi_{\text{ref}}$ 的概率比值来定义：

$$
r^*(x, y) \triangleq \beta \log \frac{\pi_\theta(y \mid x)}{\pi_{\text{ref}}(y \mid x)}
$$

这个定义一举两得：它使得奖励变得可直接计算，同时隐式地加入了一个 KL 散度约束，防止 $\pi_\theta$ 为了迎合偏好而偏离 $\pi_{\text{ref}}$ 太远，从而保证生成质量。

#### DPO 损失函数

将奖励定义代入偏好概率公式，我们就得到了一个完全由模型概率表示的人类偏好模型。机器学习的目标是最大化观测数据的对数似然，等价于最小化其负对数似然。因此，DPO 的损失函数为：

$$
\mathcal{L}_{\text{DPO}} = -\log \sigma\big(\hat{r}_\theta(y_c) - \hat{r}_\theta(y_r)\big)
$$

这本质上就是一个逻辑损失 (Logistic Loss)，它驱动模型去增大 $y_c$ 的奖励，同时减小 $y_r$ 的奖励。

---

### 构思飞跃：从“一对一”到“一对多”

重新定义问题：DPO 完美地解决了“一对一”比较。但当面对一个“好”答案 $y_c$ 和一组“坏”答案 $Y_r$ 时，我们的目标是让 $y_c$ 优于整个群组。GRPO 对此提出了一个关键的重新定义：

> “优于整个群组” 等价于 “奖励分数高于群组中那个分数最高的成员”。

$$
y_c \succ Y_r \iff r(y_c) > \max_{y_r \in Y_r} r(y_r)
$$

这个定义是 GRPO 的灵魂，它将优化目标从关注所有负样本，聚焦到了那个“最强的对手”上。

#### 借鉴 SVM 思想（最大间隔）

仅仅要求 $r(y_c)$ 更高还不够鲁棒。我们从经典机器学习算法支持向量机 (SVM) 中借鉴了“最大间隔”思想。SVM 不仅要正确分类，还追求在两类数据之间留出尽可能宽的“安全区”（Margin）。

类比：我们可以把 $y_c$ 看作正类，所有 $y_r$ 看作负类。那个奖励最高的 $y_r$ 就好比是离决策边界最近的“支持向量”。

目标：我们不仅要让 $r(y_c)$ 战胜 $\max r(y_r)$，还要让它领先一个安全间隔（margin） $\alpha$。我们的“成功条件”变为：

$$
r(y_c) \ge \max_{y_r \in Y_r} r(y_r) + \alpha
$$

#### 构建 GRPO 损失函数（Hinge Loss）

如何将这个带间隔的“成功条件”转化为损失函数？Hinge Loss（合页损失） $$\max(0, z)$$ 是完美的工具。当条件满足时，损失为 0；不满足时，损失为正。

将“成功条件”移项：

$$
\max_{y_r \in Y_r} r(y_r) - r(y_c) + \alpha \le 0
$$

当括号内的值 $z$ 大于 0 时，说明条件被违反。

于是，GRPO 的最终损失函数为：

$$
\mathcal{L}_{\text{GRPO}} = \max\Big(0, \max_{y_r \in Y_r} \hat{r}_\theta(y_r) - \hat{r}_\theta(y_c) + \alpha \Big)
$$

这里，$\hat{r}_\theta(y)$定义为：

$$
\hat{r}_\theta(y) = \beta \log \frac{\pi_\theta(y \mid x)}{\pi_{\text{ref}}(y \mid x)}
$$

- 奖励分数既是“相对偏好分”，又隐式包含 KL 散度约束。

#### 最大值梯度近似（可选）

由于 $$$\max$$$ 不可导，可使用 softmax 近似：

$$
\max_{y_r \in Y_r} \hat{r}_\theta(y_r) \approx \frac{\sum_{y_r} \hat{r}_\theta(y_r) e^{\lambda \hat{r}_\theta(y_r)}}{\sum_{y_r} e^{\lambda \hat{r}_\theta(y_r)}}
$$

- $\lambda$ 越大，近似越接近真正的最大值  
- 平滑梯度，训练更稳定

---

### 完整训练流程

1. 给定输入 $x$ 和正样本 $y_c$、负样本集合 $Y_r$。  
2. 计算奖励分数：

$$
\hat{r}_\theta(y) = \beta \log \frac{\pi_\theta(y \mid x)}{\pi_{\text{ref}}(y \mid x)}
$$

3. 找到最强负样本：

$$
y_r^* = \arg\max_{y_r \in Y_r} \hat{r}_\theta(y_r)
$$

4. 计算 GRPO 损失：

$$
\mathcal{L}_{\text{GRPO}} = \max\big(0, \hat{r}_\theta(y_r^*) - \hat{r}_\theta(y_c) + \alpha \big)
$$

5. 反向传播更新策略模型 $\pi_\theta$。  

这样，GRPO 将“一对多偏好”问题转化为可优化的 Hinge Loss，同时保留了 DPO 的 KL 散度约束和稳定训练优势。

